In [6]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [47]:

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
print('x_train: ', x_train.shape)
K = len(np.unique(y_train)) # Classes
Ntr = x_train.shape[0]
Nte = x_test.shape[0]
Din = 3072 # CIFAR10
# Din = 784 # MINIST
# Normalize pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0
mean_image = np.mean(x_train, axis=0)                   #normalizing
x_train = x_train - mean_image
x_test = x_test - mean_image
y_train = tf.keras.utils.to_categorical(y_train, num_classes=K)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=K)
x_train = np.reshape(x_train,(Ntr,Din))
x_test = np.reshape(x_test,(Nte,Din))
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
std=1e-5
w1 = std*np.random.randn(Din, K)    #weights initialization
b1 = np.zeros(K)                    #biasing
print("w1:", w1.shape)  
print("b1:", b1.shape)
batch_size = Ntr
iterations = 300    
lr = 0.2    
#lr_decay=
#reg = 
#loss_history = []
train_acc_history = []
val_acc_history = []
seed = 0
rng = np.random.default_rng(seed=seed)
#for t in range(iterations):
    #indices = np.arange(Ntr)
    #rng.shuffle(indices)
    # Forward pass
    # Backward pass
    # Printing accuracies and displaying w as images

x_train:  (50000, 32, 32, 3)
w1: (3072, 10)
b1: (10,)


In [ ]:
#defining loss function, now we have weights are biasing initiazed.
def L_i(x,y,W):
    '''
    takes in 
    '''


In [22]:
a = tf.keras.utils.to_categorical([0, 1, 2, 3], num_classes=4)
a = tf.constant(a, shape=[4, 4])
print(a)

tf.Tensor(
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]], shape=(4, 4), dtype=float32)


In [11]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)

In [80]:
x_train.shape
x_copy = x_train

In [83]:
x_copy= np.insert(x_copy,3072,,axis=1)
x_copy[0:]

ValueError: could not broadcast input array from shape (10,1) into shape (50000,1)

In [89]:
import numpy as np
N = 10
a = np.random.rand(N,N)
b = np.zeros((N,N+1))
#b[:,:-1] = a
a.shape

(10, 10)